# EECE 580G Assignment 3

## Problem 2 - Linear regression - Intro to Fairness in ML
The goal of this short problem is to use linear regression in a "real world" dataset, the boston house-prices dataset. We will also use it to introduce "fairness" problems in Machine Learning.
1. Load the dataset, split it into a train/test set, and use the Pipeline class to Standardize features (removing the mean and scaling to unit variance) and perform Linear Regression.
2. Train the pipeline and visualize the predictions vs ground truth values on the test set.

Read a little more about the boston house prices dataset below.
Features: B and LSTAT are sensitive features! In general having data about race, skin color, religion, sexual orientation, socioeconomic status, income, health, etc. is a recipe for building a biased ML system. Such features should be avoided.

A measure of (un-)fairness of a Machine Learning algorithm with respect to two subsets $Z_1$ and $Z_0$ is $ \left\lvert \frac{1}{|Z_1|} \sum_{i \in Z_1} \hat{y}_{i} - \frac{1}{|Z_0|} \sum_{i \in Z_0} \hat{y}_{i} \right\rvert $, i.e. the difference between the mean prediction in $Z_1$ and the mean prediction in $Z_0$. 

3. Consider $Z_1$ to be the samples in the test set where LSTAT is higher than the median and $Z_0$ lower than the median. Compute the fairness proxy.

4. Drop the feature B and LSTAT and retrain the pipeline and compute the 
fairness proxy, is it better? 

The fairness proxy is still pretty bad, this is because other features are directly correlated to the sensitive features. 

Consider $x_1$ and $x_2$ to be the sensitive features, a way of "orthogonalizing" the rest of the dataset is the perform the following transformation (Gram Schmidt process):
$$\begin{split}\begin{split}
v_1 & = x_1 \\
v_2 & = x_2 - \frac{x_2.v_1}{v_1.v_1}v_1\\
v_3 & = x_3 - \frac{x_3.v_1}{v_1.v_1}v_1 - \frac{x_3.v_2}{v_2.v_2}v_2\\
... \\
v_n & = x_n - \frac{x_n.v_1}{v_1.v_1}v_1 - \frac{x_n.v_2}{v_2.v_2}v_2
\end{split}\end{split}$$

The dataset $V=[v_3,...,v_n]$ is then used to train the model

5. Use the implementation provided to improve the fairness of the model. Comment on your results.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import load_boston
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [ ]:
X, y = load_boston(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=2020)
X.shape

(506, 13)

In [ ]:
X, y = load_boston(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=2020)
# 1. YOUR CODE HERE
# pipe = Pipeline([
#     ("scale", # YOUR CODE HERE),
#     ("model", # YOUR CODE HERE)
# ])

# 2. 
# YOUR CODE HERE - fit the pipe 
# YOUR CODE HERE y_hat = # predict using the pipe 

plt.scatter(y_hat, y_test, s=12)
plt.plot(y_hat, y_hat, c='gray', alpha=0.6, linewidth=1)
plt.xlabel("Predictions")
plt.ylabel("Ground truth")
plt.title('MSE = '+str(mean_squared_error(y_hat,y_test)))
plt.show()

In [ ]:
print(load_boston()['DESCR'])

In [ ]:
# 3. YOUR CODE HERE
def fairness_proxy(sensitive_feature, y_hat):
  '''
  sensitive_feature = X_test[:,-1] for LSTAT
  Z_1 are the samples in the test set where idx is higher than the median and Z_0  lower than the median
  '''
  mask = sensitive_feature > np.quantile(sensitive_feature, 0.5)
  return 0 # YOUR CODE HERE

sensitive_feature = X_test[:,-1]
print('Fairness proxy = ', fairness_proxy(sensitive_feature, y_hat))

In [ ]:
# 4. YOUR CODE HERE
# Drop B and LSTAT
# fit pipeline 
# Predict on test set
# Show scatter plot
# Show Fairness proxy 
# Comment

In [ ]:
normalized_dot = lambda a,b: a.dot(b)/(b.dot(b))

V = np.zeros_like(X)
V[:,-1] = X[:,-1]
V[:,-2] = X[:,-2] - normalized_dot(X[:,-2], V[:,-1])*V[:,-1]
for j in range(2,X.shape[1]):
  V[:,-j-1] = X[:,-j-1] - normalized_dot(X[:,-j-1], V[:,-1])*V[:,-1] - normalized_dot(X[:,-j-1], V[:,-2])*V[:,-2]

X_train_ortho, X_test_ortho, y_train, y_test = train_test_split(V[:,:-2], y, random_state=2020)

# 5. YOUR CODE HERE
# fit pipeline on X_train_ortho
# predict on X_test_ortho
# Show scatter plot
# Show Fairness proxy 
# Comment

## Problem 3 - SGD

$$ f(\textbf{x}) = \frac{1}{2}[(\eta_1 + x_1)^2 + (\eta_2 +x_2)^2] $$ where $ \eta_1,\eta_2 \sim \mathcal{N}(0,1)$


1.   What is $E[f(\textbf{x})]$?
3.   Write down the gradient of $f$
4.   Complete the code below find the minimum of $f$ using SGD with $\tau = 0.1$ and a 100 iterations
5.   Tune the learning rate $\tau(t)$ as directed by the code, comment.

In [ ]:
import numpy as np
from tqdm import tqdm_notebook as tqdm
import matplotlib.pyplot as plt 

f = lambda x, eta : ( (eta[0]+x[0])**2 + (eta[1]+x[1])**2 ) / 2 
E_f = lambda x: # YOUR CODE HERE
Grad_f = lambda x, eta : # YOUR CODE HERE

def StochasticGradDescentArray(Grad_f, x0, nbiter, tau, seed=0):
    x = x0
    xlist = [list(x0)]
    np.random.seed(seed)
    for t in range(1,nbiter+1):  
        eta = np.random.normal(size=(2)) # 1 sample of eta_1, eta_2
        x = # YOUR CODE HERE (tau is a function of t) do the gradient update  
        xlist = xlist + [list(x)]
    return np.array(xlist)

x = np.linspace(-1.5,1.5,250)
y = np.linspace(-1.5,1.5,250)

[u,v] = np.meshgrid(x,y)
F = E_f([u,v])

xarray = StochasticGradDescentArray(Grad_f,[1,1], 100, tau = lambda t: 0.1)
plt.figure(figsize=(8,8))
plt.scatter(0,0, label='Optima')
plt.contourf(x, y, F, 35)
plt.plot(xarray[:,0], xarray[:,1], 'w.-')
plt.show()

In [ ]:
powers = np.linspace(0.5,1.2,30)
errors = []
for p in tqdm(powers):
  errors_seed = []
  for s in range(100):
    xarray = StochasticGradDescentArray(Grad_f, [1,1], 100, tau = lambda t: 1/(t**p), seed=s)
    errors_seed.append(np.linalg.norm(xarray[-1]-[0,0]))
  errors.append(np.mean(errors_seed))

plt.plot(powers, errors, c='b')
plt.xlabel('Learning rate decay parameter')
plt.ylabel('Distance to optimum')
plt.show()

# Comment

## Problem 4 - MLP using tf.keras.Sequential()


### I. MNIST dataset

1.   The MNIST dataset is available in tf.keras.datasets, we load it using `load_data()`
*   Use the Sequential API to code this network:

> 1 hidden layer, size = 128, acitvation = ReLU, dropout regularization rate = 0.2.
> 
> The network outputs a probability distribution over the 10 classes (digits), this means that you need to add a softmax layer at the end of the model

*   Show `model.summary()`
*   Read the documentation of the following loss functions: `CategoricalCrossentropy`, `BinaryCrossentropy`, `SparseCategoricalCrossentropy`, and `MeanSquaredError`, which one do you choose. (Hint: look at how `y_train` is provided, you can choose 2 of these but one requires more code)    
*   Train it using the adam optimizer for 5 epochs and an appropriate loss function, you can use default learning rate, batch size, etc.
*   Evaluate it on the test set
2.   Load `x_mystery.npy`, don't look at it! Use your model to predict the digit in the image Hint: you can use the model as `model(x_mystery)` and show a bar plot of the outputs of the network
*   Now plot `x_mystery` using `imshow`, comment?
3. What does the `shuffle` function do?
*   Reshape and show a few samples 
*   Train the same model on the shuffled data
*   Comment 

### II. Fashion MNIST dataset

1.  Do the same thing (copy your code in another cell, only I.1) for the fashion_mnist dataset. Use the script provided to look at images from each class before starting the training.
2.  Save your trained model (on fashion_mnist) using the `SavedModel` format, (read the documentation of `model.save`) this will create a directory that you can reuse later. `zip` the folder and add it to your submission.


In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

# I.
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_train, x_test = x_train.reshape(x_train.shape[0], -1), x_test.reshape(x_test.shape[0], -1) # Flattening 
x_train, x_test = x_train / 255.0, x_test / 255.0 # Normalizing 

# 1. YOUR CODE HERE
def get_model():
  model = tf.keras.models.Sequential()
  # YOUR CODE HERE
  #
  # 
  return model
# model = get_model()
# model.summary()
# model.compile( # YOUR CODE )
# model.fit( # YOUR CODE )
# model.evaluate( # YOUR CODE )

# 2. YOUR CODE HERE
# Mystery
# Bar plot
# Comment
# Show mystery (don't forget to reshape to 28x28 when plotting)

def shuffle(x_train, x_test, seed=0):
  np.random.seed(seed)
  perm = np.random.permutation(x_train.shape[1])
  return x_train[:,perm], x_test[:,perm]

x_train, x_test = shuffle(x_train, x_test)

# 3. YOUR CODE HERE
# Show some samples of x_train after shuffling (don't forget to reshape to 28x28 when plotting)
# model = get_model()
# model.compile( # YOUR CODE HERE )
# model.fit( # YOUR CODE HERE )
# model.evaluate( # YOUR CODE HERE )
# Comment

11493376/11490434 [==============================] - 0s 0us/step


In [ ]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']
plt.figure(figsize=(10,10))
for i in range(25):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(x_train[i], cmap=plt.cm.binary)
    plt.xlabel(class_names[y_train[i]])
plt.show()

x_train, x_test = x_train.reshape(x_train.shape[0], -1), x_test.reshape(x_test.shape[0], -1) # Flattening 

In [ ]:
# II. YOUR CODE HERE
# model = get_model()
# model.compile( # YOUR CODE HERE )
# model.fit( # YOUR CODE HERE )
# model.evaluate( # YOUR CODE HERE )
# model.save( #YOUR CODE HERE )